In [1]:
# let's take one dataset for example
from multilab.datasets import reuter
sentences, labels = reuter()

from multilab.preprocess import Text_preprocessing
text_preprocessing = Text_preprocessing()

dataframe = text_preprocessing.labels_to_dataframe(sentences,labels)
preprocessded_dataset = text_preprocessing.initial_preprocess(dataframe, chunk_value = 25)
dataset, frequency_list = text_preprocessing.keep_labels(preprocessded_dataset,keep_ratio=0.10)
slice_dataset = text_preprocessing.dataset_slice(dataset,ratio=0.25)

import numpy as np
all_sente = list(slice_dataset['text'])
all_label = np.array(slice_dataset.drop('text', 1))

X_train, X_test, y_train, y_test = text_preprocessing.split_dataset(all_sente, all_label)

X_trains,sequ  = text_preprocessing.pad_sentences(sentences = X_train)
X_tests ,sequb = text_preprocessing.pad_sentences(sentences = X_test)

from multilab.models import Elmo_Word_Model

config = {
                         'no_of_labels'               :  y_train.shape[1],
                         'learning_rate'              : 0.001,
                         'rnn_units'                  : 100,
                         'epoch'                      : 1,
                         'batch_size'                 : 12,
                         'dropout'                    : 0.2,
                         'output_type'                : 'state_output',
                         'train_elmo'                 : True,
                         'result_path'                : '/Users/monk/Desktop',
                        }


el_m = Elmo_Word_Model(X_train, y_train, X_test,  y_test, config)
el_m.train()

[nltk_data] Downloading package reuters to /Users/monk/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
lower_case done
punctuation removed
text cleaning done


100%|██████████| 12/12 [00:00<00:00, 7330.56it/s]

([['bellsouth', 'ltbls', 'completes', 'purchase', 'for', 'shares', 'bellsouth', 'corp', 'said', 'it', 'completed', 'its', 'previouslyannounced', 'agreement', 'to', 'acquire', 'ltdataserv', 'inc', 'for', 'an', 'exchange', 'of', 'shares', 'bellsouth', 'said'], ['science', 'management', 'corp', 'ltsmg', 'th', 'qtr', 'oper', 'net', 'oper', 'shr', 'profit', 'cts', 'vs', 'loss', 'cts', 'oper', 'net', 'profit', 'vs', 'loss', 'revs', 'mln', 'vs', 'mln', 'avg'], ['shultz', 'predicts', 'rapid', 'narrowing', 'of', 'us', 'trade', 'gap', 'secretary', 'of', 'state', 'george', 'shultz', 'said', 'the', 'us', 'would', 'erase', 'its', 'large', 'foreign', 'trade', 'deficit', 'faster', 'than'], ['james', 'river', 'corp', 'ltjr', 'sets', 'regular', 'dividend', 'qtly', 'div', 'cts', 'vs', 'cts', 'prior', 'pay', 'april', 'record', 'april', '', '', '', '', '', '', '', ''], ['safetykleen', 'ltsk', 'to', 'buy', 'stake', 'in', 'oil', 'refiner', 'safetykleen', 'corp', 'said', 'it', 'has', 'tentatively', 'agreed',

InternalError: Unable to get element as bytes.

In [ ]:
from multilab.models import Elmo_Word_Model

config = {
                         'no_of_labels'               :  y_train.shape[1],
                         'learning_rate'              : 0.001,
                         'rnn_units'                  : 100,
                         'epoch'                      : 1,
                         'batch_size'                 : 12,
                         'dropout'                    : 0.2,
                         'output_type'                : 'state_output',
                         'train_elmo'                 : True,
                         'result_path'                : '/Users/monk/Desktop',
                        }


el_m = Elmo_Word_Model(X_train, y_train, X_test,  y_test, config)
el_m.train()

In [ ]:
# testing

In [ ]:
import tensorflow as tf 
import tensorflow_hub as hub

ins = tf.placeholder(tf.string, (None), name='pl_phrase_text')


module = hub.Module('https://tfhub.dev/google/elmo/2', trainable=True)
embeddings = module(dict(text=ins))
embeddings = tf.expand_dims(embeddings, axis=1)

In [ ]:
with tf.Session() as sess:
    sess.run([tf.global_variables_initializer(), tf.tables_initializer()])
    print(sess.run(embeddings,feed_dict={ins:['hello']}).shape)

In [ ]:
import tensorflow as tf 
import tensorflow_hub as hub

tokens = tf.placeholder(tf.string, (None), name='pl_phrase_text')
pl_phrase_len = tf.placeholder(tf.int32, (None), name='pl_phrase_len')

tokens_input = [["the", "cat", "is", "on", "the", "mat"],
                ["dogs", "are", "in", "the", "fog", ""]]
tokens_length = [6, 5]


module = hub.Module('https://tfhub.dev/google/elmo/2', trainable=True)
module_features = module(dict(tokens=tokens, sequence_len = pl_phrase_len),
                                 signature='tokens', as_dict=True)
embeddingse = module_features["elmo"]

with tf.variable_scope('Layer1'):
    cell_fw1 = tf.nn.rnn_cell.LSTMCell(num_units=128, state_is_tuple=True)
    cell_bw1 = tf.nn.rnn_cell.LSTMCell(num_units=128, state_is_tuple=True)

    outputs1, states1 = tf.nn.bidirectional_dynamic_rnn(
        cell_fw=cell_fw1,
        cell_bw=cell_bw1,
        inputs=embeddingse,
        dtype=tf.float32)

In [ ]:
with tf.Session() as sess:
    sess.run([tf.global_variables_initializer(), tf.tables_initializer()])
    out, data,t = sess.run([outputs1,embeddingse,module_features],feed_dict={tokens: tokens_input, pl_phrase_len: tokens_length})
    
    
    
    

In [ ]:
rnn_output = tf.reshape(outputs1[0], (-1, 128))
rnn_output.shape

In [ ]:
rnn_outputs = tf.reshape(outputs1[0], (-1, 128 * 6))

In [ ]:
rnn_outputs.shape

In [ ]:
from tqdm import tqdm
import nltk

# padded_sequence = [i + [0] * (max_sequence - len(i)) if len(i) < max_sequence else i for i in batch_data_j]


sentences = ['check this out','i am very happy', 'what is', ['check','this'],'he a a a abs']


def pad_sentences(self, sentences, sequence_len):
    padded_sentences = []
    actual_length    = []


    for sentence in tqdm(sentences):
        if not isinstance(sentence, list):
            token = nltk.word_tokenize(sentence)
        else:
            token = sentence
        
        if len(token) < sequene_len:
            actual_length.append(len(token))
            token = token + [''] * (sequene_len-len(token))
        else:
            actual_length.append(len(token))
        
        padded_sentences.append(token)
    return padded_sentences, actual_length

In [ ]:
chy = pad_sentences(sentences,5)

In [ ]:
chy

In [ ]:
actual_len = []
for m in chy[0]:
    actu = [k for k in m if k!='']
    actual_len.append(len(actu))
    

In [ ]:
import numpy as np

def actual_len(list_data):
    
    actual_ = []
    for sequence in list_data:
        actual = [sub_ for sub_ in sequence if sub_!='']
        actual_.append(len(actual))
    return actual_
        
    
    

In [ ]:
data        = actual_len(chy[0])
second_list = actual_len(chy[0])
data.extend(second_list)

In [ ]:
import tensorflow as tf
import numpy as np
import os
import tensorflow_hub as hub

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False
try:
    from tensorflow.python.util import module_wrapper as deprecation
except ImportError:
    from tensorflow.python.util import deprecation_wrapper as deprecation
deprecation._PER_MODULE_WARNING_LIMIT = 0



class Elmo_word_model(object):
    
    def __init__(self, 
                no_of_labels,
                learning_rate, 
                rnn_units, 
                train_elmo = True, 
                output_type = 'state_output', 
                max_sentence_words = 4):

        tf.reset_default_graph()

        # feature extraction network  ------------------------------------------>>
        
        # pass raw string 
        # one hot labels
        sentences             = tf.placeholder(tf.string, (None), name='sentences')
        self.targets          = tf.placeholder(tf.int32, [None, None], name='labels' )
        sequence_length       = tf.placeholder(tf.int32, (None), name='sequence_len')

        keep_prob             = tf.placeholder(tf.float32, name='dropout')



        self.placeholders     = {
                                'sentence': sentences, 
                                'labels': self.targets, 
                                'drop': keep_prob, 
                                'sequence_length': sequence_length
                                }

        module                = hub.Module('https://tfhub.dev/google/elmo/2', trainable = train_elmo )
        module_features       = module(dict(tokens=sentences, sequence_len = sequence_length),
                                 signature='tokens', as_dict=True)
        embeddings            = module_features["elmo"]



        # sequence learning network -------------------------------------------------------->
         #bilstm model
        with tf.variable_scope('forward'):
            fr_cell = tf.contrib.rnn.LSTMCell(num_units = rnn_units)
            dropout_fr = tf.contrib.rnn.DropoutWrapper(fr_cell, output_keep_prob = 1. - keep_prob)
            
        with tf.variable_scope('backward'):
            bw_cell = tf.contrib.rnn.LSTMCell(num_units = rnn_units)
            dropout_bw = tf.contrib.rnn.DropoutWrapper(bw_cell, output_keep_prob = 1. - keep_prob)
            
        with tf.variable_scope('encoder') as scope:
            model,last_state = tf.nn.bidirectional_dynamic_rnn(dropout_fr,
                                                               dropout_bw,
                                                               inputs = embeddings,
                                                               dtype=tf.float32)

        
        if output_type == 'flat':

            logits = tf.reshape(model[0], (-1, rnn_units * max_sentence_words))
            # dense layer with xavier weights
            fc_layer = tf.get_variable(name='fully_connected',
                                    shape=[rnn_units * max_sentence_words, no_of_labels],
                                    dtype=tf.float32,
                                    initializer=tf.contrib.layers.xavier_initializer())
            
            # bias 
            bias    = tf.get_variable(name='bias',
                                    shape=[no_of_labels],
                                    dtype=tf.float32,
                                    initializer=tf.contrib.layers.xavier_initializer())
            
            #final output 
            self.x_ = tf.add(tf.matmul(logits,fc_layer),bias)

        else:

            logits = tf.concat([last_state[0].c,last_state[1].c],axis=-1)
             # dense layer with xavier weights
            fc_layer = tf.get_variable(name='fully_connected',
                                    shape=[2*rnn_units, no_of_labels],
                                    dtype=tf.float32,
                                    initializer=tf.contrib.layers.xavier_initializer())
            
            # bias 
            bias    = tf.get_variable(name='bias',
                                    shape=[no_of_labels],
                                    dtype=tf.float32,
                                    initializer=tf.contrib.layers.xavier_initializer())
            
            #final output 
            self.x_ = tf.add(tf.matmul(logits,fc_layer),bias)


         #optimization and loss calculation ---------------------------------->>
        
        self.cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(logits = self.x_, labels = tf.cast(self.targets,tf.float32))
        self.loss = tf.reduce_mean(tf.reduce_sum(self.cross_entropy, axis=1))
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.loss)
        self.predictions = tf.cast(tf.sigmoid(self.x_) > 0.5, tf.int32)

In [ ]:
def model_execute(model):
    with tf.Session() as sess:
        sess.run([tf.global_variables_initializer(), tf.tables_initializer()])
        
        model_out, train = sess.run([model.loss, model.optimizer],
                                    feed_dict={model.placeholders['sentence']:[['hello','how','are','you'],['dep','as','','']],
                                               model.placeholders['labels']:np.random.randint(0,2,[2,4]), 
                                               model.placeholders['sequence_length']:[4,2],model.placeholders['drop']:0.0})
        print(model_out)
        
        
if __name__=='__main__':
    
    model_out_ = Elmo_word_model(4,0.001,10)
    model_execute(model_out_)

In [ ]:
# no_of_labels,
#                 learning_rate, 
#                 rnn_units, 
#                 train_elmo = True, 
#                 output_type = 'state_output', 
#                 max_sentence_words = 4

In [ ]:
from tqdm import tqdm
import nltk

def pad_sentences(self, sentences):
    

    padded_sentences = []
    actual_length    = []

    sentences = [seq.split() for seq in sentences]
    sequence_len = max(list(map(len, sentences)))
    print(sequence_len)


    for sentence in tqdm(sentences):
        if not isinstance(sentence, list):
            token = nltk.word_tokenize(sentence)
        else:
            token = sentence

        if len(token) < sequence_len:
            actual_length.append(len(token))
            token = token + [''] * (sequence_len-len(token))
        else:
            actual_length.append(len(token))

        padded_sentences.append(token)
    return padded_sentences, actual_length

In [ ]:
pad_sentences(['check gh','ell','af'])